In [ ]:
#Karam Issa ;)

!pip install opencv-python
!pip install Pillow
# This will download the raw file from the opencv githup repo, to garuantee the file will always be uploaded correctly
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml

This code sets the KAGGLE_CONFIG_DIR environment variable to /content, which is a directory in the Google Colab environment. The KAGGLE_CONFIG_DIR environment variable is used by the Kaggle API client to locate the Kaggle configuration file that contains the user's Kaggle API credentials.

By setting the KAGGLE_CONFIG_DIR environment variable to /content, you are telling the Kaggle API client to look for the Kaggle configuration file in the /content directory, which is where you can store the configuration file in the Colab environment. This is necessary for the Kaggle API client to work properly in Colab.


**Dont forget to add your kaggle.json api key 

In [7]:
#In the context of looping through a folder of images, we can use the os.listdir() function to get a list of all the files in the folder, and then use os.path.join() to construct the full path to each image file. This makes it easy to read and manipulate the images in the folder.
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [ ]:
!kaggle datasets download -d vin1234/count-the-number-of-faces-present-in-an-image

In [9]:
!chmod 600 /content/kaggle.json

In [ ]:
!unzip \*.zip && rm*.zip

In [11]:
# Import the necessary libraries

# Used for opening and manipulating image files
from PIL import Image

# Used for makinf HTTP requests to web servers as the input image if from a URL
import requests

# Used for working with input and output streams.
from io import BytesIO
# Opencv Libraray fro image processing and computer vission
import cv2

# Used here to convert the image to/from a PIL Image object from/to a NumPy array.
import numpy as np

#The csv module provides classes for reading and writing CSV files, such as csv.reader, csv.writer, csv.DictReader, and csv.DictWriter. These classes provide methods and attributes for working with CSV files, such as iterating over rows, accessing columns by name, writing rows
import csv

This code iterates through each image file in the directory specified by image_path. It attempts to open each image file using the PIL library and skips over any files that cannot be opened. For each valid image file, it converts the image to a NumPy array that can be processed by OpenCV, loads a face detection classifier from a specified file path, and detects faces in the image using the detectMultiScale method. Finally, it writes a new row to the output CSV file with the filename and number of faces detected.

In [19]:

# Set the path to the directory containing the images
image_path = '/content/train/image_data/'


# Open the output CSV file for writing   
with open('output.csv', mode='w') as output_file:

  # Create a CSV writer object
  csv_writer = csv.writer(output_file)

  # Loop through each image file in the directory      
  for img in range(10001, 18208):
    
    # Construct the full path to the image file
    image_filename = str(img) + '.jpg'
    full_path = image_path + image_filename


    # Attempt to open the image file
    try:
      image = Image.open(full_path)
    except:
      # If there is an error opening the image file, skip it and move on to the next file
      continue
    else:
      # Convert the image to a NumPy array that can be processed by OpenCv
      image_array = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)



      # Load the face detection classifier from the specified file path
      face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
           

      # Convert the image to grayscale, which is required for face detection
      gray_image = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)

      # Detect faces in the image using the face detection classifier
      faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.05, minNeighbors=5)

      # Write a new row to the output CSV file with the filename and number of faces detected
      csv_writer.writerow([image_filename, len(faces)])

            


In [46]:
# Open the input file for reading
with open('output.csv', 'r') as input_file:
    reader = csv.reader(input_file)

    # Read in the data rows
    data = [row for row in reader]

# Insert a new header row at the beginning of the data
header_row = ['Name', 'HeadCount']
data.insert(0, header_row)

# Open the output file for writing
with open('output.csv', 'w', newline='') as output_file:
    writer = csv.writer(output_file)

    # Write out the data to the output file
    for row in data:
        writer.writerow(row)

This code compares the headcount values for each image in the train.csv file with the headcount values obtained from running face detection on the images and stored in the output.csv file.

It does this by creating a dictionary headcount_dict that stores the headcount values for each file. It first reads through the train.csv file to get the filename and headcount value for each image. It then reads through the output.csv file to get the headcount value obtained from running face detection on each image. If a match is found between the filename in train.csv and output.csv, the headcount values for both are stored in the headcount_dict dictionary.

Finally, the code prints out the headcount values for each file in the headcount_dict dictionary. If a file is not found in the output.csv file, a message is printed indicating that the file was not found.

In [ ]:
# Open the train.csv file for reading
with open('/content/train/train.csv', 'r') as train_file:
    train_reader = csv.DictReader(train_file)

    # Open the output.csv file for reading
    with open('output.csv', 'r') as output_file:
        output_reader = csv.DictReader(output_file)

        # Create a dictionary to store the headcount values for each file
        headcount_dict = {}

        # Iterate over the rows in the train.csv file
        for train_row in train_reader:

            # Get the filename from the train row
            filename = train_row['Name']

            # Check if the filename is in the output.csv file
            for output_row in output_reader:
                if output_row['Name'] == filename:

                    # Store the headcount value for this file in the dictionary
                    headcount_dict[filename] = {
                        'train': int(train_row['HeadCount']),
                        'output': int(output_row['HeadCount'])
                    }

                    # Reset the output reader to the beginning of the file
                    output_file.seek(0)
                    next(output_reader)

                    # Break out of the inner loop since we found a match
                    break

            # If the filename wasn't found in the output.csv file, print a message
            else:
                print(f'File {filename} not found in output.csv')

# Print the headcount values for each file
for filename, counts in headcount_dict.items():
    print(f'{filename}: train={counts["train"]}, output={counts["output"]}')

In [67]:
# Compare the headcount values for each file
same_count = 0
for counts in headcount_dict.values():
    if counts['train'] == counts['output']:
        same_count += 1

print(f'{same_count} files had the same train and output headcount values')
print(str(same_count/5733 *100)+'% of the files precisely detected correctly')

1284 files had the same train and output headcount values
22.39665096807954% of the files precisely detected correctly


In [68]:
# Compare the headcount values for each file
same_count = 0
for counts in headcount_dict.values():
    if counts['train'] == counts['output'] or (counts['train'] == counts['output'] +1):
        same_count += 1

print(f'{same_count} files had output headcount values (-1), one face missing from the train headcount values')
print(str(same_count/5733 *100)+'% of the files were partially detected correctly with an possible error of maximum of 1 missing face per image file not detected')

2594 files had output headcount values (-1), one face missing from the train headcount values
45.246816675388104% of the files were partially detected correctly with an possible error of maximum of 1 missing face per image file not detected


In [69]:
# Compare the headcount values for each file
same_count = 0
for counts in headcount_dict.values():
    if counts['train'] == counts['output'] or (counts['train'] == counts['output'] +1) or (counts['train'] == counts['output'] +2):
        same_count += 1

print(f'{same_count} files had output headcount values (-1), one face missing from the train headcount values')
print(str(same_count/5733 *100)+'% of the files were partially detected correctly with an possible error of  maximum 2 missing face per image file not detected')

3489 files had output headcount values (-1), one face missing from the train headcount values
60.858189429618% of the files were partially detected correctly with an possible error of  maximum 2 missing face per image file not detected


In [70]:
# Create lists of true and predicted headcounts
from sklearn.metrics import f1_score
y_true = [v['train'] for v in headcount_dict.values()]
y_pred = [v['output'] for v in headcount_dict.values()]

# Calculate the F1 score
f1 = f1_score(y_true, y_pred, average='weighted')

# Print the F1 score
print(f'The F1 score is: {f1:.2f}')

The F1 score is: 0.24


This code block reads data from two CSV files, train.csv and output.csv, and writes the output to a new CSV file named output_with_percentage.csv.

The code first creates a dictionary called headcount_dict to store the headcount values for each file. It then iterates over the rows in train.csv, retrieves the filename, and checks if that file exists in output.csv. If it does, the code stores the headcount values for that file in headcount_dict. If it doesn't, the code prints a message indicating that the file was not found.

After the data is read from both files and stored in headcount_dict, the code calculates the percentage of faces detected for each file by dividing the output headcount by the train headcount and multiplying by 100. The resulting percentage is then written to a new CSV file called output_with_percentage.csv, along with the other data from output.csv.

The csv.DictReader() and csv.DictWriter() functions are used to read and write the CSV files, respectively. The seek() function is used to reset the output file reader to the beginning of the file so that it can be read again for each filename in the train.csv file.

In [43]:

with open('/content/train/train.csv', 'r') as train_file, \
        open('output.csv', 'r') as output_file, \
        open('output_with_percentage.csv', 'w', newline='') as output_with_percentage_file:

    train_reader = csv.DictReader(train_file)
    output_reader = csv.DictReader(output_file)

    # Create a dictionary to store the headcount values for each file
    headcount_dict = {}

    # Iterate over the rows in the train.csv file
    for train_row in train_reader:
        # Get the filename from the train row
        filename = train_row['Name']

        # Check if the filename is in the output.csv file
        for output_row in output_reader:
            if output_row['Name'] == filename:
                # Store the headcount value for this file in the dictionary
                headcount_dict[filename] = {
                    'train': int(train_row['HeadCount']),
                    'output': int(output_row['HeadCount'])
                }

                # Reset the output reader to the beginning of the file
                output_file.seek(0)
                next(output_reader)

                # Break out of the inner loop since we found a match
                break

        # If the filename wasn't found in the output.csv file, print a message
        else:
            print(f'File {filename} not found in output.csv')

    # Calculate the percentage of faces detected for each file and write to output_with_percentage.csv
    fieldnames = ['Name', 'HeadCount', 'percentageOfFacesDetected']
    writer = csv.DictWriter(output_with_percentage_file, fieldnames=fieldnames)
    writer.writeheader()

    for row in output_reader:
        filename = row['Name']
        counts = headcount_dict.get(filename)
        if counts:
            percentage = counts['output'] / counts['train'] * 100
            row['percentageOfFacesDetected'] = percentage
            writer.writerow(row)



This code block reads data from a CSV file named output_with_percentage.csv and calculates the mean percentage of faces detected across all files in the CSV.

The code first opens the CSV file for reading using the with statement and the csv.DictReader() function. It then extracts the values of the percentageOfFacesDetected column from each row of the CSV file and appends them to a list called percentages.

After all values have been extracted, the code uses the statistics.mean() function to calculate the mean of the percentages list. The resulting mean is stored in a variable called mean_percentage.

Finally, the code prints the mean percentage of faces detected using an f-string that includes the value of mean_percentage.

In [44]:
import statistics

# Open the output_with_percentage.csv file for reading
with open('output_with_percentage.csv', 'r') as output_file:
    output_reader = csv.DictReader(output_file)

    # Extract the values of the percentageOfFacesDetected column
    percentages = []
    for row in output_reader:
        percentages.append(float(row['percentageOfFacesDetected']))

    # Calculate the mean of the percentages
    mean_percentage = statistics.mean(percentages)

    # Print the mean percentage
    print(f"Mean percentage of faces detected: {mean_percentage}")

Mean percentage of faces detected: 79.12311646553822
